# Find word boundaries in BH

This seq2seq model finds word boundaries in consonantal texts in BH. It makes word boundary predictions on the book of Jonah, after being trained on the rest of the MT.

In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

from sklearn.utils import shuffle

C:\Users\geitb\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from tf.app import use
A = use('bhsa', hoist=globals())
A.displaySetup(extraFeatures='g_cons')

TF app is up-to-date.
Using annotation/app-bhsa commit 806f4fc237348be26ec06ac86e3e2c5888504753 (=latest)
  in C:\Users\geitb/text-fabric-data/__apps__/bhsa.
Using etcbc/bhsa/tf - c r1.5 in C:\Users\geitb/text-fabric-data
Using etcbc/phono/tf - c r1.2 in C:\Users\geitb/text-fabric-data
Using etcbc/parallels/tf - c r1.2 in C:\Users\geitb/text-fabric-data


**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="provenance of BHSA = Biblia Hebraica Stuttgartensia Amstelodamensis">BHSA</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Writing/Hebrew" title="('Hebrew characters and transcriptions',)">Character table</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/c/0_home.html" title="BHSA feature documentation">Feature docs</a> <a target="_blank" href="https://github.com/annotation/app-bhsa" title="bhsa API documentation">bhsa API</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Api/Fabric/" title="text-fabric-api">Text-Fabric API 7.3.15</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Use/Search/" title="Search Templates Introduction and Reference">Search Reference</a>

A train and test set are defined. The model is trained on all the books of the MT, except Jonah. The model will be used to predict parts of speech for this book.

The data are prepared.

In [3]:
def prepare_train_data():
    """"
    
    """

    input_chunks = []
    output_chunks = []
    input_vocab = set()
    output_vocab = {'x', '\t', '\n', 'w'}
    all_chars = set()
    
    for word in F.otype.s('word'):
        for char in (F.g_cons.v(word)):
            all_chars.add(char)
    all_chars.add(' ')
    all_chars.add('־')

    for bo in F.otype.s("book"): 
        
        # exclude Jonah from train data
        if F.book.v(bo) == "Jona":
            continue
        
        words_in_book = L.d(bo, "word")
        
        # prepare the train input
        # add space if g_cons has value '' (empty), often ketiv/qere issues like the BNJ in Judges 20:13
        train_text = "".join(["".join([F.g_cons.v(wo), F.trailer_utf8.v(wo)]) if (F.g_cons.v(wo) != '' or F.lex.v(wo) == 'H') else ' ' for wo in words_in_book])
        train_text = "".join([char for char in train_text if char in all_chars])
        train_text = train_text.replace("  ", " ")
        #train_text = train_text.replace("־ ", " ")
        train_split = train_text.split(' ')
        for unit in range(len(train_split)-5):
            input_chunk = ''
            for piece in range(unit, unit+5):
                input_chunk += train_split[piece]
                input_chunk += ' '
            input_chunks.append(input_chunk.strip())

            for char in input_chunk:
                input_vocab.add(char)
            
        # prepare the train output 
        train_out = ''.join([''.join([''.join(['x' for char in F.g_cons.v(wo)]), 'w', F.trailer_utf8.v(wo)]) if (F.g_cons.v(wo) != '' or F.lex.v(wo) == 'H') else ' ' for wo in words_in_book ])
        train_out2 = ''.join([char for char in train_out if char in {'x', 'w', ' '}]).strip()
        train_out2 = train_out2.replace("  ", " ")
        train_out_split = train_out2.split(' ')
        for unit in range(len(train_out_split)-4):
            output_chunk = ''
            for piece in range(unit, unit+5):
                output_chunk += train_out_split[piece]
                #output_chunk += ' '
                
            output_chunk2 = ['\t']
            for elem in output_chunk.strip():
                output_chunk2.append(elem)
            output_chunk2.append('\n')
            output_chunks.append(output_chunk2)
        
    
    input_chars = sorted(list(input_vocab))
    output_vocab = sorted(list(output_vocab))
    
    max_len_input = max([len(clause) for clause in input_chunks])
    max_len_output = max([len(output_phr) for output_phr in output_chunks])
    
    # shuffle the data
    #input_clauses, output_phrases = shuffle(input_clauses, output_phrases)
    
    return input_chunks, output_chunks, input_vocab, output_vocab, max_len_input, max_len_output, all_chars

In [4]:
def prepare_test_data(all_chars):
    """
    The function returns:
    input_chunks, a list containing the text of chunks in Jonah
    """
    input_chunks = []
    output_chunks = []
    
    for bo in F.otype.s("book"): 

        if F.book.v(bo) != "Jona":
            continue
        
        words_in_book = L.d(bo, "word")

        # prepare the test input
        jona_text = "".join(["".join([F.g_cons.v(wo), F.trailer_utf8.v(wo)]) if (F.g_cons.v(wo) != '') else (' ' if F.lex.v(wo) != 'H' else '') for wo in words_in_book])

        jona_text = "".join([char for char in jona_text if char in all_chars])
        jona_text = jona_text.replace("  ", " ")
        jona_split = jona_text.split(' ')
        for unit in range(len(jona_split)-5):
            input_chunk = ''
            for piece in range(unit, unit+5):
                input_chunk += jona_split[piece]
                input_chunk += ' '
            input_chunks.append(input_chunk.strip())

        # prepare test output
        test_out = ''.join([''.join([''.join(['x' for char in F.g_cons.v(wo)]), 'w', F.trailer_utf8.v(wo)]) if (F.g_cons.v(wo) != '' or F.lex.v(wo) == 'H') else ' ' for wo in words_in_book ])
        test_out2 = ''.join([char for char in test_out if char in {'x', 'w', ' '}]).strip()
        test_out2 = test_out2.replace("  ", " ")
        test_out_split = test_out2.split(' ')
        for unit in range(len(test_out_split)-4):
            output_chunk = ''
            for piece in range(unit, unit+5):
                output_chunk += test_out_split[piece]
                
            output_chunk2 = ['\t']
            for elem in output_chunk.strip():
                output_chunk2.append(elem)
            output_chunk2.append('\n')
            output_chunks.append(output_chunk2)
            
        # prepare test output, word nodes per chunk
        output_n_chunks = []
        test_nodes = ''.join([''.join([str(wo), 'w', F.trailer_utf8.v(wo)]) if (F.g_cons.v(wo) != '' or F.lex.v(wo) == 'H') else ' ' for wo in words_in_book ])
        test_nodes2 = ''.join([char for char in test_nodes if char in {'1' ,'2','3','4','5','6','7','8','9','0', 'w', ' '}]).strip()
        test_nodes2 = test_nodes2.replace("  ", " ")
        test_nodes_split = test_nodes2.split(' ')
        
        for unit in range(len(test_nodes_split)-4):
            output_n_chunk = []
            for piece in range(unit, unit+5):
                output_n_chunk.append(test_nodes_split[piece])
                
            output_n_chunk = ''.join((output_n_chunk)) 
            output_n_chunk = output_n_chunk.split('w')[:-1]
            output_n_chunk = [int(wo) for wo in output_n_chunk]

            output_n_chunks.append(output_n_chunk)
        
    
    return input_chunks, output_chunks, output_n_chunks

In [7]:
def create_dicts(input_vocab, output_vocab):
    """
    The network can only handle numeric data. This function provides four dicts. 
    Two of them map between integers and the input characters (one dict for every direction), the other two 
    map between integers and parts of speech.
    """

    
    input_idx2char = {}
    input_char2idx = {}

    for k, v in enumerate(input_chars):
        input_idx2char[k] = v
        input_char2idx[v] = k
        
    output_idx2char = {}
    output_char2idx = {}
    
    for k, v in enumerate(output_vocab):
        output_idx2char[k] = v
        output_char2idx[v] = k
     
    
    return input_idx2char, input_char2idx, output_idx2char, output_char2idx

In [8]:
def one_hot_encode(nb_samples, max_len_input, max_len_output, input_chars, output_vocab, input_char2idx, output_char2idx, input_clauses, output_pos):
    """
    Categorical data are generally one-hot encoded in neural networks, which is done here.
    
    """

    tokenized_input_data = np.zeros(shape = (nb_samples,max_len_input,len(input_chars)), dtype='float32')
    tokenized_output = np.zeros(shape = (nb_samples,max_len_output,len(output_vocab)), dtype='float32')
    target_data = np.zeros((nb_samples, max_len_output, len(output_vocab)),dtype='float32')

    for i in range(nb_samples):
        for k, ch in enumerate(input_clauses[i]):
            tokenized_input_data[i, k, input_char2idx[ch]] = 1
        
        for k, ch in enumerate(output_pos[i]):
            tokenized_output[i, k, output_char2idx[ch]] = 1

            # decoder_target_data will be ahead by one timestep and will not include the start character.
            if k > 0:
                target_data[i, k-1, output_char2idx[ch]] = 1
                
    return tokenized_input_data, tokenized_output, target_data

In [9]:
def define_LSTM_model(input_chars, output_vocab):
    """
    
    
    """

    # Encoder model

    encoder_input = Input(shape=(None,len(input_chars)))
    encoder_LSTM = LSTM(256,activation = 'relu',return_state = True, return_sequences=True)(encoder_input)
    encoder_LSTM = LSTM(256,return_state = True)(encoder_LSTM)
    encoder_outputs, encoder_h, encoder_c = encoder_LSTM
    encoder_states = [encoder_h, encoder_c]
    
    # Decoder model

    decoder_input = Input(shape=(None,len(output_vocab)))
    decoder_LSTM = LSTM(256, return_sequences=True, return_state = True)
    decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
    decoder_dense = Dense(len(output_vocab), activation='softmax')
    decoder_out = decoder_dense (decoder_out)
    
    model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])

    model.summary()

    return encoder_input, encoder_states, decoder_input, decoder_LSTM, decoder_dense, model

In [10]:
def compile_and_train(model, tokenized_input, tokenized_output, batch_size, epochs, validation_split):

    model.compile(optimizer='adam', loss='categorical_crossentropy')
    model.fit(x=[tokenized_input,tokenized_output], 
              y=target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=validation_split)
    
    return model

In [11]:
nb_samples = 261000

In [12]:
input_clauses, output_pos, input_chars, output_vocab, max_len_input, max_len_output, all_chars = prepare_train_data()
input_idx2char, input_char2idx, output_idx2char, output_char2idx = create_dicts(input_chars, output_vocab)
tokenized_input, tokenized_output, target_data = one_hot_encode(nb_samples, max_len_input, max_len_output, input_chars, output_vocab, input_char2idx, output_char2idx, input_clauses, output_pos)

In [30]:
print(len(input_clauses), len(output_pos))

261768 261768


In [13]:
test_clauses, output_test, output_nodes = prepare_test_data(all_chars)
tokenized_test_data, _, _ = one_hot_encode(len(test_clauses), max_len_input, max_len_output, input_chars, output_vocab, input_char2idx, output_char2idx, test_clauses, output_pos)

In [14]:
print(len(test_clauses), len(output_test))

580 580


In [24]:
encoder_input, encoder_states, decoder_input, decoder_LSTM, decoder_dense, model = define_LSTM_model(input_chars, output_vocab)
model = compile_and_train(model, tokenized_input, tokenized_output, 512, 70, 0.1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, None, 26)     0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   [(None, None, 256),  289792      input_9[0][0]                    
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, None, 4)      0                                            
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, 256), (None, 525312      lstm_7[0][0]                     
                                                                 lstm_7[0][1]                     
          

Epoch 56/70
234900/234900 [==============================] - 123s 523us/step - loss: 0.0091 - val_loss: 0.1419
Epoch 57/70
234900/234900 [==============================] - 124s 530us/step - loss: 0.0096 - val_loss: 0.0102
Epoch 58/70
234900/234900 [==============================] - 125s 533us/step - loss: 0.0054 - val_loss: 0.0106
Epoch 59/70
234900/234900 [==============================] - 126s 537us/step - loss: 0.0082 - val_loss: 0.0103
Epoch 60/70
234900/234900 [==============================] - 126s 537us/step - loss: 0.0050 - val_loss: 0.0106
Epoch 61/70
234900/234900 [==============================] - 124s 527us/step - loss: 0.0049 - val_loss: 0.0109
Epoch 62/70
234900/234900 [==============================] - 128s 545us/step - loss: 0.0081 - val_loss: 0.1045
Epoch 63/70
234900/234900 [==============================] - 127s 541us/step - loss: 0.0078 - val_loss: 0.0104
Epoch 64/70
234900/234900 [==============================] - 124s 527us/step - loss: 0.0042 - val_loss: 0.0107
E

In [25]:
# Inference models for testing

# Encoder inference model
encoder_model_inf = Model(encoder_input, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [26]:
def decode_seq(inp_seq):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(output_vocab)))
    target_seq[0, 0, output_char2idx['\t']] = 1
    
    pred_pos = []
    stop_condition = False
    
    while not stop_condition:
        
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_out_char = output_idx2char[max_val_index]
        pred_pos.append(sampled_out_char)
        
        if (sampled_out_char == '\n'):
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(output_vocab)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
        
    return pred_pos

In [27]:
correct = 0

for seq_index in range(len(output_test)):
    inp_seq = tokenized_test_data[seq_index:seq_index+1]
    
    pred_pos = decode_seq(inp_seq)

    if output_test[seq_index][1:-1] == pred_pos[:-1]:
        correct += 1
    else:
         print('-')
         print(seq_index)
         print('Input clause:', test_clauses[seq_index])
         print('Predicted word boundaries:', pred_pos[:-1])
         print('Input chunk:', output_test[seq_index][1:-1])  
        
print(correct)


-
39
Input clause: >L־HJM WJHJ S<R־GDWL BJM WH>NJH
Predicted word boundaries: ['x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w']
Input chunk: ['x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'w']
-
40
Input clause: WJHJ S<R־GDWL BJM WH>NJH XCBH
Predicted word boundaries: ['x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w']
Input chunk: ['x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w']
-
41
Input clause: S<R־GDWL BJM WH>NJH XCBH LHCBR
Predicted word boundaries: ['x', 'x', 'x', 'w'

-
165
Input clause: BCLJ HS<R HGDWL HZH <LJKM
Predicted word boundaries: ['x', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w']
Input chunk: ['x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w']
-
187
Input clause: N>BDH BNPC H>JC HZH W>L־TTN
Predicted word boundaries: ['x', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'w']
Input chunk: ['x', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'w']
-
188
Input clause: BNPC H>JC HZH W>L־TTN <LJNW
Predicted word boundaries: ['x', 'w', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', '

-
296
Input clause: QDCK MCMRJM HBLJ־CW> XSDM J<ZBW
Predicted word boundaries: ['x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w']
Input chunk: ['x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w']
-
297
Input clause: MCMRJM HBLJ־CW> XSDM J<ZBW W>NJ
Predicted word boundaries: ['x', 'w', 'x', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w']
Input chunk: ['x', 'x', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w']
-
308
Input clause: JCW<TH LJHWH WJ>MR JHWH LDG
Predicted word boundaries: ['x', 'x', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', '

-
424
Input clause: KJ־CBW MDRKM HR<H WJNXM H>LHJM
Predicted word boundaries: ['x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w']
Input chunk: ['x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w']
-
468
Input clause: MMNJ KJ VWB MWTJ MXJJ
Predicted word boundaries: ['x', 'x', 'x', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w']
Input chunk: ['x', 'x', 'x', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'w']
-
469
Input clause: KJ VWB MWTJ MXJJ WJ>MR
Predicted word boundaries: ['x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'w']
Input chunk: ['x', 'x', 'w', 'x', 'x', 'x', 'w', 'x'

-
543
Input clause: >LHJM >L־JWNH HHJVB XRH־LK <L־HQJQJWN
Predicted word boundaries: ['x', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'w']
Input chunk: ['x', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'x', 'w']
-
558
Input clause: L>־<MLT BW WL> GDLTW CBN־LJLH
Predicted word boundaries: ['x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w']
Input chunk: ['x', 'x', 'w', 'x', 'x', 'x', 'x', 'w', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'x', 'w', 'x', 'w', 'x', 'x', 'w', 'x', 'x', 'x', 'x', 'w']
-
559
Input clause: BW WL> GDLTW CBN־LJLH HJH
Predicted word boundaries: ['x', 'x', 'w', 'x', 'w',

In [28]:
correct/len(output_test)

0.8327586206896552

In [29]:
correct = 0

for seq_index in range(len(output_test)):
    inp_seq = tokenized_test_data[seq_index:seq_index+1]
    
    pred_pos = decode_seq(inp_seq)

    print('-')
    print(seq_index)
    print('Input clause:', test_clauses[seq_index])
    print(''.join(pred_pos[:-1]), 'predicted')
    print(''.join(output_test[seq_index][1:-1]), 'true value')  
    if ''.join(pred_pos[:-1]) == ''.join(output_test[seq_index][1:-1]):
        print('identical')
        correct += 1
print(correct)

-
0
Input clause: WJHJ DBR־JHWH >L־JWNH BN־>MTJ L>MR
xwxxxwxxxwxxxxwxxwxxxxwxxwxxxxwxwxxxw predicted
xwxxxwxxxwxxxxwxxwxxxxwxxwxxxxwxwxxxw true value
identical
-
1
Input clause: DBR־JHWH >L־JWNH BN־>MTJ L>MR QWM
xxxwxxxxwxxwxxxxwxxwxxxxwxwxxxwxxxw predicted
xxxwxxxxwxxwxxxxwxxwxxxxwxwxxxwxxxw true value
identical
-
2
Input clause: >L־JWNH BN־>MTJ L>MR QWM LK
xxwxxxxwxxwxxxxwxwxxxwxxxwxxw predicted
xxwxxxxwxxwxxxxwxwxxxwxxxwxxw true value
identical
-
3
Input clause: BN־>MTJ L>MR QWM LK >L־NJNWH
xxwxxxxwxwxxxwxxxwxxwxxwxxxxxw predicted
xxwxxxxwxwxxxwxxxwxxwxxwxxxxxw true value
identical
-
4
Input clause: L>MR QWM LK >L־NJNWH H<JR
xwxxxwxxxwxxwxxwxxxxxwxwxxxw predicted
xwxxxwxxxwxxwxxwxxxxxwxwxxxw true value
identical
-
5
Input clause: QWM LK >L־NJNWH H<JR HGDWLH
xxxwxxwxxwxxxxxwxwxxxwxwxxxxxw predicted
xxxwxxwxxwxxxxxwxwxxxwxwxxxxxw true value
identical
-
6
Input clause: LK >L־NJNWH H<JR HGDWLH WQR>
xxwxxwxxxxxwxwxxxwxwxxxxxwxwxxxw predicted
xxwxxwxxxxxwxwxxxwxwxxxxxwxwxxxw true value
id

-
57
Input clause: M<LJHM WJWNH JRD >L־JRKTJ HSPJNH
xwxxxxxwxwxxxxwxxxwxxwxxxxxwxwxxxxxw predicted
xwxxxxxwxwxxxxwxxxwxxwxxxxxwxwxxxxxw true value
identical
-
58
Input clause: WJWNH JRD >L־JRKTJ HSPJNH WJCKB
xwxxxxwxxxwxxwxxxxxwxwxxxxxwxwxxxxw predicted
xwxxxxwxxxwxxwxxxxxwxwxxxxxwxwxxxxw true value
identical
-
59
Input clause: JRD >L־JRKTJ HSPJNH WJCKB WJRDM
xxxwxxwxxxxxwxwxxxxxwxwxxxxwxwxxxxw predicted
xxxwxxwxxxxxwxwxxxxxwxwxxxxwxwxxxxw true value
identical
-
60
Input clause: >L־JRKTJ HSPJNH WJCKB WJRDM WJQRB
xxwxxxxxwxwxxxxxwxwxxxxwxwxxxxwxwxxxxw predicted
xxwxxxxxwxwxxxxxwxwxxxxwxwxxxxwxwxxxxw true value
identical
-
61
Input clause: HSPJNH WJCKB WJRDM WJQRB >LJW
xwxxxxxwxwxxxxwxwxxxxwxwxxxxwxxxxw predicted
xwxxxxxwxwxxxxwxwxxxxwxwxxxxwxxxxw true value
identical
-
62
Input clause: WJCKB WJRDM WJQRB >LJW RB
xwxxxxwxwxxxxwxwxxxxwxxxxwxxw predicted
xwxxxxwxwxxxxwxwxxxxwxxxxwxxw true value
identical
-
63
Input clause: WJRDM WJQRB >LJW RB HXBL
xwxxxxwxwxxxxwxxxxwxxwxwxxxw predicted
xwxx

-
117
Input clause: W>T־JHWH >LHJ HCMJM >NJ JR>
xwxxwxxxxwxxxxwxwxxxxwxxxwxxxw predicted
xwxxwxxxxwxxxxwxwxxxxwxxxwxxxw true value
identical
-
118
Input clause: >LHJ HCMJM >NJ JR> >CR־<FH
xxxxwxwxxxxwxxxwxxxwxxxwxxxw predicted
xxxxwxwxxxxwxxxwxxxwxxxwxxxw true value
identical
-
119
Input clause: HCMJM >NJ JR> >CR־<FH >T־HJM
xwxxxxwxxxwxxxwxxxwxxxwxxwxwxxw predicted
xwxxxxwxxxwxxxwxxxwxxxwxxwxwxxw true value
identical
-
120
Input clause: >NJ JR> >CR־<FH >T־HJM W>T־HJBCH
xxxwxxxwxxxwxxxwxxwxwxxwxwxxwxwxxxxw predicted
xxxwxxxwxxxwxxxwxxwxwxxwxwxxwxwxxxxw true value
identical
-
121
Input clause: JR> >CR־<FH >T־HJM W>T־HJBCH WJJR>W
xxxwxxxwxxxwxxwxwxxwxwxxwxwxxxxwxwxxxxxw predicted
xxxwxxxwxxxwxxwxwxxwxwxxwxwxxxxwxwxxxxxw true value
identical
-
122
Input clause: >CR־<FH >T־HJM W>T־HJBCH WJJR>W H>NCJM
xxxwxxxwxxwxwxxwxwxxwxwxxxxwxwxxxxxwxwxxxxxw predicted
xxxwxxxwxxwxwxxwxwxxwxwxxxxwxwxxxxxwxwxxxxxw true value
identical
-
123
Input clause: >T־HJM W>T־HJBCH WJJR>W H>NCJM JR>H
xxwxwxxwxwxxwxwx

xxxxwxxwxwxxwxxxxwxwxxxw true value
identical
-
176
Input clause: KJ HJM HWLK WS<R <LJHM
xxwxwxxwxxxxwxwxxxwxxxxxw predicted
xxwxwxxwxxxxwxwxxxwxxxxxw true value
identical
-
177
Input clause: HJM HWLK WS<R <LJHM WJQR>W
xwxxwxxxxwxwxxxwxxxxxwxwxxxxxw predicted
xwxxwxxxxwxwxxxwxxxxxwxwxxxxxw true value
identical
-
178
Input clause: HWLK WS<R <LJHM WJQR>W >L־JHWH
xxxxwxwxxxwxxxxxwxwxxxxxwxxwxxxxw predicted
xxxxwxwxxxwxxxxxwxwxxxxxwxxwxxxxw true value
identical
-
179
Input clause: WS<R <LJHM WJQR>W >L־JHWH WJ>MRW
xwxxxwxxxxxwxwxxxxxwxxwxxxxwxwxxxxxw predicted
xwxxxwxxxxxwxwxxxxxwxxwxxxxwxwxxxxxw true value
identical
-
180
Input clause: <LJHM WJQR>W >L־JHWH WJ>MRW >NH
xxxxxwxwxxxxxwxxwxxxxwxwxxxxxwxxxw predicted
xxxxxwxwxxxxxwxxwxxxxwxwxxxxxwxxxw true value
identical
-
181
Input clause: WJQR>W >L־JHWH WJ>MRW >NH JHWH
xwxxxxxwxxwxxxxwxwxxxxxwxxxwxxxxw predicted
xwxxxxxwxxwxxxxwxwxxxxxwxxxwxxxxw true value
identical
-
182
Input clause: >L־JHWH WJ>MRW >NH JHWH >L־N>
xxwxxxxwxwxxxxxwxxxwxxxxwxx

-
233
Input clause: >LHJW MM<J HDGH WJ>MR QR>TJ
xxxxxwxwxxxwxwxxxwxwxxxxwxxxxxw predicted
xxxxxwxwxxxwxwxxxwxwxxxxwxxxxxw true value
identical
-
234
Input clause: MM<J HDGH WJ>MR QR>TJ MYRH
xwxxxwxwxxxwxwxxxxwxxxxxwxxxxw predicted
xwxxxwxwxxxwxwxxxxwxxxxxwxwxxxw true value
-
235
Input clause: HDGH WJ>MR QR>TJ MYRH LJ
xwxxxwxwxxxxwxxxxxwxxxxwxxw predicted
xwxxxwxwxxxxwxxxxxwxwxxxwxxw true value
-
236
Input clause: WJ>MR QR>TJ MYRH LJ >L־JHWH
xwxxxxwxxxxxwxxxxwxxwxxwxxxxw predicted
xwxxxxwxxxxxwxwxxxwxxwxxwxxxxw true value
-
237
Input clause: QR>TJ MYRH LJ >L־JHWH WJ<NNJ
xxxxxwxxxxwxxwxxwxxxxwxwxxxxxw predicted
xxxxxwxwxxxwxxwxxwxxxxwxwxxxxxw true value
-
238
Input clause: MYRH LJ >L־JHWH WJ<NNJ MBVN
xwxxxwxxwxxwxxxxwxwxxxxxwxwxxxw predicted
xwxxxwxxwxxwxxxxwxwxxxxxwxwxxxw true value
identical
-
239
Input clause: LJ >L־JHWH WJ<NNJ MBVN C>WL
xxwxxwxxxxwxwxxxxxwxwxxxwxxxxw predicted
xxwxxwxxxxwxwxxxxxwxwxxxwxxxxw true value
identical
-
240
Input clause: >L־JHWH WJ<NNJ MBVN C>WL CW<TJ
xxwxx

-
294
Input clause: TPLTJ >L־HJKL QDCK MCMRJM HBLJ־CW>
xxxxxwxxwxxxxwxxxxwxwxxxxxwxxxxwxxxw predicted
xxxxxwxxwxxxxwxxxxwxxxxxxwxxxxwxxxw true value
-
295
Input clause: >L־HJKL QDCK MCMRJM HBLJ־CW> XSDM
xxwxxxxwxxxxwxwxxxxxwxxxxwxxxwxxxxw predicted
xxwxxxxwxxxxwxxxxxxwxxxxwxxxwxxxxw true value
-
296
Input clause: QDCK MCMRJM HBLJ־CW> XSDM J<ZBW
xxxxwxwxxxxxwxxxxwxxxwxxxxwxxxxxw predicted
xxxxwxxxxxxwxxxxwxxxwxxxxwxxxxxw true value
-
297
Input clause: MCMRJM HBLJ־CW> XSDM J<ZBW W>NJ
xwxxxxxwxxxxwxxxwxxxxwxxxxxwxwxxxw predicted
xxxxxxwxxxxwxxxwxxxxwxxxxxwxwxxxw true value
-
298
Input clause: HBLJ־CW> XSDM J<ZBW W>NJ BQWL
xxxxwxxxwxxxxwxxxxxwxwxxxwxwxxxw predicted
xxxxwxxxwxxxxwxxxxxwxwxxxwxwxxxw true value
identical
-
299
Input clause: XSDM J<ZBW W>NJ BQWL TWDH
xxxxwxxxxxwxwxxxwxwxxxwxxxxw predicted
xxxxwxxxxxwxwxxxwxwxxxwxxxxw true value
identical
-
300
Input clause: J<ZBW W>NJ BQWL TWDH >ZBXH־LK
xxxxxwxwxxxwxwxxxwxxxxwxxxxxwxxw predicted
xxxxxwxwxxxwxwxxxwxxxxwxxxxxwxxw true value
iden

-
353
Input clause: WJQR> WJ>MR <WD >RB<JM JWM
xwxxxxwxwxxxxwxxxwxxxxxxwxxxw predicted
xwxxxxwxwxxxxwxxxwxxxxxxwxxxw true value
identical
-
354
Input clause: WJ>MR <WD >RB<JM JWM WNJNWH
xwxxxxwxxxwxxxxxxwxxxwxwxxxxxw predicted
xwxxxxwxxxwxxxxxxwxxxwxwxxxxxw true value
identical
-
355
Input clause: <WD >RB<JM JWM WNJNWH NHPKT
xxxwxxxxxxwxxxwxwxxxxxwxxxxxw predicted
xxxwxxxxxxwxxxwxwxxxxxwxxxxxw true value
identical
-
356
Input clause: >RB<JM JWM WNJNWH NHPKT WJ>MJNW
xxxxxxwxxxwxwxxxxxwxxxxxwxwxxxxxxw predicted
xxxxxxwxxxwxwxxxxxwxxxxxwxwxxxxxxw true value
identical
-
357
Input clause: JWM WNJNWH NHPKT WJ>MJNW >NCJ
xxxwxwxxxxxwxxxxxwxwxxxxxxwxxxxw predicted
xxxwxwxxxxxwxxxxxwxwxxxxxxwxxxxw true value
identical
-
358
Input clause: WNJNWH NHPKT WJ>MJNW >NCJ NJNWH
xwxxxxxwxxxxxwxwxxxxxxwxxxxwxxxxxw predicted
xwxxxxxwxxxxxwxwxxxxxxwxxxxwxxxxxw true value
identical
-
359
Input clause: NHPKT WJ>MJNW >NCJ NJNWH B>LHJM
xxxxxwxwxxxxxxwxxxxwxxxxxwxwxxxxxw predicted
xxxxxwxwxxxxxxwxxxxwxxxxxwxwxxxx

-
408
Input clause: HR<H WMN־HXMS >CR BKPJHM MJ־JWD<
xwxxxwxwxxwxwxxxwxxxwxwxxxxxwxxwxxxxw predicted
xwxxxwxwxxwxwxxxwxxxwxwxxxxxwxxwxxxxw true value
identical
-
409
Input clause: WMN־HXMS >CR BKPJHM MJ־JWD< JCWB
xwxxwxwxxxwxxxwxwxxxxxwxxwxxxxwxxxxw predicted
xwxxwxwxxxwxxxwxwxxxxxwxxwxxxxwxxxxw true value
identical
-
410
Input clause: >CR BKPJHM MJ־JWD< JCWB WNXM
xxxwxwxxxxxwxxwxxxxwxxxxwxwxxxw predicted
xxxwxwxxxxxwxxwxxxxwxxxxwxwxxxw true value
identical
-
411
Input clause: BKPJHM MJ־JWD< JCWB WNXM H>LHJM
xwxxxxxwxxwxxxxwxxxxwxwxxxwxwxxxxxw predicted
xwxxxxxwxxwxxxxwxxxxwxwxxxwxwxxxxxw true value
identical
-
412
Input clause: MJ־JWD< JCWB WNXM H>LHJM WCB
xxwxxxxwxxxxwxwxxxwxwxxxxxwxwxxw predicted
xxwxxxxwxxxxwxwxxxwxwxxxxxwxwxxw true value
identical
-
413
Input clause: JCWB WNXM H>LHJM WCB MXRWN
xxxxwxwxxxwxwxxxxxwxwxxwxwxxxxw predicted
xxxxwxwxxxwxwxxxxxwxwxxwxwxxxxw true value
identical
-
414
Input clause: WNXM H>LHJM WCB MXRWN >PW
xwxxxwxwxxxxxwxwxxwxwxxxxwxxxw predicted
xwxxxwxw

-
464
Input clause: W<TH JHWH QX־N> >T־NPCJ MMNJ
xwxxxwxxxxwxxwxxwxxwxxxxwxxxxw predicted
xwxxxwxxxxwxxwxxwxxwxxxxwxxxxw true value
identical
-
465
Input clause: JHWH QX־N> >T־NPCJ MMNJ KJ
xxxxwxxwxxwxxwxxxxwxxxxwxxw predicted
xxxxwxxwxxwxxwxxxxwxxxxwxxw true value
identical
-
466
Input clause: QX־N> >T־NPCJ MMNJ KJ VWB
xxwxxwxxwxxxxwxxxxwxxwxxxw predicted
xxwxxwxxwxxxxwxxxxwxxwxxxw true value
identical
-
467
Input clause: >T־NPCJ MMNJ KJ VWB MWTJ
xxwxxxxwxxxxwxxwxxxwxxxxw predicted
xxwxxxxwxxxxwxxwxxxwxxxxw true value
identical
-
468
Input clause: MMNJ KJ VWB MWTJ MXJJ
xxxxwxxwxxxwxxxxwxxxxw predicted
xxxxwxxwxxxwxxxxwxwxxxw true value
-
469
Input clause: KJ VWB MWTJ MXJJ WJ>MR
xxwxxxwxxxxwxxxxwxwxxxxw predicted
xxwxxxwxxxxwxwxxxwxwxxxxw true value
-
470
Input clause: VWB MWTJ MXJJ WJ>MR JHWH
xxxwxxxxwxxxxwxwxxxxwxxxxw predicted
xxxwxxxxwxwxxxwxwxxxxwxxxxw true value
-
471
Input clause: MWTJ MXJJ WJ>MR JHWH HHJVB
xxxxwxxxxwxwxxxxwxxxxwxwxxxxw predicted
xxxxwxwxxxwxwxxxxwxxxxwxwxxxxw t

-
527
Input clause: RWX QDJM XRJCJT WTK HCMC
xxxwxxxxwxxxxxxwxwxxwxwxxxw predicted
xxxwxxxxwxxxxxxwxwxxwxwxxxw true value
identical
-
528
Input clause: QDJM XRJCJT WTK HCMC <L־R>C
xxxxwxxxxxxwxwxxwxwxxxwxxwxxxw predicted
xxxxwxxxxxxwxwxxwxwxxxwxxwxxxw true value
identical
-
529
Input clause: XRJCJT WTK HCMC <L־R>C JWNH
xxxxxxwxwxxwxwxxxwxxwxxxwxxxxw predicted
xxxxxxwxwxxwxwxxxwxxwxxxwxxxxw true value
identical
-
530
Input clause: WTK HCMC <L־R>C JWNH WJT<LP
xwxxwxwxxxwxxwxxxwxxxxwxwxxxxxw predicted
xwxxwxwxxxwxxwxxxwxxxxwxwxxxxxw true value
identical
-
531
Input clause: HCMC <L־R>C JWNH WJT<LP WJC>L
xwxxxwxxwxxxwxxxxwxwxxxxxwxwxxxxw predicted
xwxxxwxxwxxxwxxxxwxwxxxxxwxwxxxxw true value
identical
-
532
Input clause: <L־R>C JWNH WJT<LP WJC>L >T־NPCW
xxwxxxwxxxxwxwxxxxxwxwxxxxwxxwxxxxw predicted
xxwxxxwxxxxwxwxxxxxwxwxxxxwxxwxxxxw true value
identical
-
533
Input clause: JWNH WJT<LP WJC>L >T־NPCW LMWT
xxxxwxwxxxxxwxwxxxxwxxwxxxxwxwxxxw predicted
xxxxwxwxxxxxwxwxxxxwxxwxxxxwxwxxxw true va